In [1]:
# Inicjalizacja
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

     |████████████████████████████████| 4.3MB 2.7MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
# Importy
import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

from hyperopt import hp, fmin, tpe, STATUS_OK, STATUS_FAIL

import eli5
from eli5.sklearn import PermutationImportance

In [4]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


In [8]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

## Feature Engineering

In [0]:
# "Numeryzujemy" parametry tekstowe
SUFFIX_CAT = '__cat'

for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [0]:
# Parametry liczbowe czyścimy ze śmieci typu "m3" czy "KM"
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split(' ')[0]))
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int( str(x).split('cm')[0].replace(' ','')))

In [0]:
# Definiujemy model
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [24]:
# Ustalamy parametry i uruchamiamy model
feats = ['param_napęd__cat', 'param_rok-produkcji', 'param_stan__cat', 'param_skrzynia-biegów__cat', 'param_faktura-vat__cat', 'param_moc', 'param_marka-pojazdu__cat', 'feature_kamera-cofania__cat', 'param_typ__cat', 'param_pojemność-skokowa', 'seller_name__cat', 'feature_wspomaganie-kierownicy__cat', 'param_model-pojazdu__cat', 'param_wersja__cat', 'param_kod-silnika__cat', 'feature_system-start-stop__cat', 'feature_asystent-pasa-ruchu__cat', 'feature_czujniki-parkowania-przednie__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_regulowane-zawieszenie__cat']

xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0
}

run_model(xgb.XGBRegressor(**xgb_params), feats)

[21:44:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:44:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:44:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)

In [25]:
# Alternatywne parametry dla modelu
xgb_params_alt = {
    'max_depth': 6,
    'n_estimators': 200,
    'learning_rate': 0.2,
    'seed': 0
}
run_model(xgb.XGBRegressor(**xgb_params_alt), feats)

[21:45:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:45:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:45:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-7878.041183329359, 51.651643267930815)

## Hyperopt

In [34]:
# Funkcja
def obj_func(params):
  print("Trenuje z parametrami: ")
  print(params)

  try:
    mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)
    return {'loss': np.abs(mean_mae), 'status': STATUS_OK}
  except (RuntimeError, TypeError, NameError):
    print("Ała")
  else:
    return {'status': STATUS_FAIL}

# Parametry
xgb_reg_params = { 
    'learning_rate':     hp.choice('learning_rate',         np.arange(0.02, 0.3, 0.02)),
    'max_depth':         hp.choice('max_depth',             np.arange(9, 14, 1, dtype=int)),
    'subsample':         hp.quniform('subsample',           0.5, 1, 0.05),
    'colsample_bytree':  hp.quniform('colsample_bytree',     0.8, 1, 0.02),
    'objective':         'reg:squarederror',
    'n_estimators':     100,
    'seed':              0,
}

# Działaj!
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=5)

best  

Trenuje z parametrami: 
{'colsample_bytree': 0.92, 'learning_rate': 0.5700000000000001, 'max_depth': 12, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.6000000000000001}
Trenuje z parametrami: 
{'colsample_bytree': 0.97, 'learning_rate': 1.02, 'max_depth': 11, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9500000000000001}
Trenuje z parametrami: 
{'colsample_bytree': 0.81, 'learning_rate': 2.6999999999999997, 'max_depth': 12, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.65}
Trenuje z parametrami: 
{'colsample_bytree': 0.99, 'learning_rate': 0.18, 'max_depth': 12, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9}
Trenuje z parametrami: 
{'colsample_bytree': 0.89, 'learning_rate': 1.41, 'max_depth': 11, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.7000000000000001}
Trenuje z parametrami: 
{'colsample_bytree': 0.81, 'lear

{'colsample_bytree': 0.87,
 'learning_rate': 5,
 'max_depth': 3,
 'subsample': 0.9500000000000001}

In [43]:
# Funkcja
def obj_func(params):
  print("Trenuje z parametrami: ")
  print(params)

  try:
    mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)
    return {'loss': np.abs(mean_mae), 'status': STATUS_OK}
  except (RuntimeError, TypeError, NameError):
    print("Ała zdefiniowane")
    return {'status': STATUS_FAIL}
  else:
    print("Ała niezdefiniowane")
    return {'status': STATUS_FAIL}

# Parametry
xgb_reg_params = { 
    'learning_rate':     hp.choice('learning_rate',         np.arange(0.02, 0.3, 0.02)),
    'max_depth':         hp.choice('max_depth',             np.arange(5, 9, 1, dtype=int)),
    'subsample':         hp.quniform('subsample',           0.5, 1, 0.05),
    'colsample_bytree':  hp.quniform('colsample_bytree',     0.8, 1, 0.02),
    'objective':         'reg:squarederror',
    'n_estimators':      150,
    'seed':              0,
}

# Działaj!
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)

best  

Trenuje z parametrami: 
{'colsample_bytree': 0.88, 'learning_rate': 0.02, 'max_depth': 5, 'n_estimators': 150, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.6000000000000001}
Trenuje z parametrami: 
{'colsample_bytree': 0.86, 'learning_rate': 0.19999999999999998, 'max_depth': 7, 'n_estimators': 150, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.7000000000000001}
Trenuje z parametrami: 
{'colsample_bytree': 0.9400000000000001, 'learning_rate': 0.19999999999999998, 'max_depth': 6, 'n_estimators': 150, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.65}
Trenuje z parametrami: 
{'colsample_bytree': 0.88, 'learning_rate': 0.24, 'max_depth': 8, 'n_estimators': 150, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9500000000000001}
Trenuje z parametrami: 
{'colsample_bytree': 1.0, 'learning_rate': 0.19999999999999998, 'max_depth': 6, 'n_estimators': 150, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.75}
Trenuje z parametrami: 
{'cols

{'colsample_bytree': 0.9,
 'learning_rate': 11,
 'max_depth': 3,
 'subsample': 1.0}